In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from plotly.offline import iplot, init_notebook_mode
import cufflinks as cf

In [ ]:
df = pd.read_csv('/kaggle/input/stroke-prediction-dataset/healthcare-dataset-stroke-data.csv')

In [ ]:
df.head()

In [ ]:
df.isna().sum()

In [ ]:
df.info()

In [ ]:
df.describe()

# Some age are less than 1...let's change that to 1 as we can't have age less than one

In [ ]:
plt.figure(figsize = (50,10))
sns.countplot(data=df, x = 'age')

In [ ]:
len(df[df['age'] <= 1])

In [ ]:
def childage(col):
    for i in col:
        
        if i <= 1:
            return 1
        else:
            return i

In [ ]:
df['age'] = df[['age']].apply(childage,axis=1)

In [ ]:
df.describe()

# LET's dive into bmi as some of its data are missing

In [ ]:
df.isna().sum()

In [ ]:
df.corr()['bmi'].sort_values()[1:-1]

In [ ]:
sns.heatmap(data=df.corr() , cmap = 'plasma' , annot=True)

# Okay we'll keep this for later, let's now focus on all the object categorical columns and make dummy variables

In [ ]:
df.head()

In [ ]:
df2 = pd.get_dummies(df ,columns = ['gender' , 'ever_married' , 'work_type' , 'Residence_type' , 'smoking_status'] , drop_first= True)

In [ ]:
df2.head()

In [ ]:
df = df2

In [ ]:
df2.drop('id',axis=1,inplace=True)

In [ ]:
df2.info()

In [ ]:
df2['smoking_status_never smoked'] =  df2['smoking_status_never smoked'].astype(np.int64)
df2['smoking_status_smokes'] =  df2['smoking_status_smokes'].astype(np.int64)

In [ ]:
df2['smoking_status_formerly smoked'] =  df2['smoking_status_formerly smoked'].astype(np.int64)

In [ ]:
df2.info()

In [ ]:
df.head()

In [ ]:
df2['gender_Male'] =  df2['gender_Male'].astype(np.int64)
df2['gender_Other'] =  df2['gender_Other'].astype(np.int64)
df2['ever_married_Yes'] =  df2['ever_married_Yes'].astype(np.int64)
df2['work_type_Never_worked'] =  df2['work_type_Never_worked'].astype(np.int64)
df2['work_type_Private'] =  df2['work_type_Private'].astype(np.int64)
df2['work_type_Self-employed'] =  df2['work_type_Self-employed'].astype(np.int64)
df2['work_type_children'] =  df2['work_type_children'].astype(np.int64)
df2['Residence_type_Urban'] =  df2['Residence_type_Urban'].astype(np.int64)

In [ ]:
df2.info()

In [ ]:
data = pd.read_csv('/kaggle/input/stroke-prediction-dataset/healthcare-dataset-stroke-data.csv')
# just to keep referencing from point to point

In [ ]:
df2.describe()

In [ ]:
df2.drop('stroke',axis=1,inplace=True)
df2['stroke'] = data['stroke']
df2.head()

# LET'S GET BACK TO MISSING DATA IN BMI COLUMN

In [ ]:
df2.isna().sum()

In [ ]:
df2.corr()['bmi'].sort_values()[:-1]

In [ ]:
201 / len(df2)

In [ ]:
data.head()

In [ ]:
sns.pairplot(data=data)

In [ ]:
sns.boxplot(data=data , x = 'hypertension',y='bmi' , hue = 'heart_disease')

In [ ]:
df2['age'].describe()

In [ ]:
def impute_age(age):
    if age in range(0,5):
        return 1
    elif age in range(5,10):
        return 2
    elif age in range(10,15):
        return 3
    elif age in range(15,20):
        return 4
    elif age in range(20,25):
        return 5
    elif age in range(25,30):
        return 6
    elif age in range(30,35):
        return 7
    elif age in range(35,40):
        return 8
    elif age in range(40,45):
        return 9
    elif age in range(45,50):
        return 10
    elif age in range(50,55):
        return 11
    elif age in range(55,60):
        return 12
    elif age in range(60,65):
        return 13
    elif age in range(65,70):
        return 14
    elif age in range(70,75):
        return 15
    elif age in range(75,80):
        return 16
    elif age in range(80,85):
        return 17
    

In [ ]:
df2['age_category'] = df['age'].apply(impute_age)

In [ ]:
df2.corr()['bmi'][:-1].sort_values()

In [ ]:
plt.figure(figsize=(20,10))
sns.boxplot(data=df2, x = 'age_category',y='bmi' , hue = 'hypertension')

In [ ]:
for i in range(1,18):
    print(f'for category {i} , if married YES, the median bmi is : ')
    print(df2[(df2['age_category']==i) & (df2['ever_married_Yes'] == 1)]['bmi'].median())
    print(f'for category {i} , if married NO, the median bmi is : ')
    print(df2[(df2['age_category']==i) & (df2['ever_married_Yes'] == 0)]['bmi'].median())
    i += 1
# GOOD ANALYSIS...but not that useful as the difference between bmi of married and unmarried is not that high..max is only 4
# Hence , we'll resort to excluding the marital condition 

In [ ]:
for i in range(1,18):
    print(f'for category {i} , if HYPERTENSION YES, the median bmi is : ')
    print(df2[(df2['age_category']==i) & (df2['hypertension'] == 1)]['bmi'].median())
    print(f'for category {i} , if HYPER TENSION NO, the median bmi is : ')
    print(df2[(df2['age_category']==i) & (df2['hypertension'] == 0)]['bmi'].median())
    i += 1    
# IMPORTANT!!!!!!!

# Only hypertension seems out to  be a good metric...tried others as well but none had "significant" variations in the category

In [ ]:
def bmi_imputer(cols):
    bmi = cols[0]
    age_cat = cols[1]
    ht = cols[2]
    if pd.isnull(bmi):
        if age_cat == 1:
            return 18
        elif age_cat == 2:
            return 18.3
        elif age_cat == 3:
            return 21.8
        elif ((age_cat == 4) & (ht == 1)):
            return 63.3
        elif ((age_cat == 4) & (ht == 0)):
            return 24.2
        elif ((age_cat == 5) & (ht == 1)):
            return 37.65
        elif ((age_cat == 5) & (ht == 0)):
            return 26.2
        elif ((age_cat == 6) & (ht == 1)):
            return 37.5
        elif ((age_cat == 6) & (ht == 0)):
            return 27
        elif ((age_cat == 7) & (ht == 1)):
            return 45.1
        elif ((age_cat == 7) & (ht == 0)):
            return 29.5
        elif ((age_cat == 8) & (ht == 1)):
            return 37.2
        elif ((age_cat == 8) & (ht == 0)):
            return 29.25
        elif ((age_cat == 9) & (ht == 1)):
            return 37.65
        elif ((age_cat == 9) & (ht == 0)):
            return 30
        elif ((age_cat == 10) & (ht == 1)):
            return 35.5
        elif ((age_cat == 10) & (ht == 0)):
            return 29.6
        elif ((age_cat == 11) & (ht == 1)):
            return 32.05
        elif ((age_cat == 11) & (ht == 0)):
            return 30.2
        elif ((age_cat == 12) & (ht == 1)):
            return 33.2
        elif ((age_cat == 12) & (ht == 0)):
            return 30
        elif ((age_cat == 13) & (ht == 1)):
            return 33.75
        elif ((age_cat == 13) & (ht == 0)):
            return 30.1
        elif ((age_cat == 14) & (ht == 1)):
            return 31.1
        elif ((age_cat == 14) & (ht == 0)):
            return 29
        elif ((age_cat == 15) & (ht == 1)):
            return 31.1
        elif ((age_cat == 15) & (ht == 0)):
            return 29.25
        elif ((age_cat == 16) & (ht == 1)):
            return 29
        elif ((age_cat == 16) & (ht == 0)):
            return 27.85
        elif ((age_cat == 17) & (ht == 1)):
            return 28.55
        elif ((age_cat == 17) & (ht == 0)):
            return 27.35
    else:
        return bmi
            

In [ ]:
df2['bmi'] = df2[['bmi' , 'age_category' , 'hypertension']].apply(bmi_imputer , axis=1)

In [ ]:
df2.isna().sum()

In [ ]:
df2.info()

In [ ]:
df = df2.copy()

In [ ]:
df.info()

In [ ]:
df.drop('age_category' , axis=1 , inplace=True)

In [ ]:
df.info()

In [ ]:
df.dropna(inplace=True)

In [ ]:
df.info()

# SEEMS LIKE DATASET HAS BEEN MORE OR LESS CLEANED!!

In [ ]:
df.tail(10)

# LET'S CHECK WHETHER THE DATA DISTRIBUTION OF THE DEPENDENT VARIABLE IS NORMAL OR NOT, DEPENDING ON THAT WE'LL CHOOSE STANDARDIZATION METHOD

In [ ]:
sns.histplot(data=df, x = 'bmi', kde=True)
# Left skewed ...not normal distribution

In [ ]:
sns.histplot(data=df, x = 'age', kde=True)
# definitely not normal distribution

In [ ]:
sns.histplot(data=df, x = 'avg_glucose_level', kde=True)
# not a normal distribution

# NONE OF OUR CONTINOUS VARIABLE ARE NORMALLY DISTRIBUTED; HENCE, WE'LL APPLY STANDARD SCALAR

In [ ]:
X = df.iloc[: , :-1]
y = df.iloc[: , -1]

In [ ]:
from sklearn.model_selection import train_test_split

# SINCE DATA IS HIGHLY UNBALANCED, (ONLY 5% VALUES HAVE STROKE=1), WE'LL DO A BIT OF DATA MANIPULATION

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

# LET'S TRY REMAKING A DATASET OUT OF df WHICH FOLLOWS A TREND SIMILAR TO df BUT HAS LESS VALUES OF STROKE=0 AND THE RETAINING THE ROWNS WITH STROKE=1

In [ ]:
len(df[df['stroke']==0])

In [ ]:
df_0 = df[df['stroke']==0]
df_1 = df[df['stroke']==1]

In [ ]:
df_0 = df_0.sample(1250 , random_state=42)

In [ ]:
df_new = pd.concat([df_0,df_1] , axis=0)

In [ ]:
df_new.reset_index(inplace=True)

In [ ]:
df_new.drop('index',axis=1,inplace=True)

In [ ]:
df_new.head()

In [ ]:
A = df_new.drop('stroke',axis=1)
B = df_new['stroke']
A_train,A_test,B_train,B_test = train_test_split(A,B,test_size=0.2,random_state=42)
A_train = scaler.fit_transform(A_train)
A_test = scaler.transform(A_test)

#  RANDOM FOREST

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report,confusion_matrix,f1_score

In [ ]:
rf_f1 = []
for i in range(1,101):
    rf_new = RandomForestClassifier(n_estimators=i ,random_state=42)
    rf_new.fit(A_train,B_train)
    rf_new_pred = rf_new.predict(scaler.transform(X))
    rf_f1.append(f1_score(y,rf_new_pred))

In [ ]:
rf_df = pd.DataFrame({'Trees': np.arange(1,101) , 'f1':rf_f1})
rf_df[rf_df['f1']==rf_df['f1'].max()]

In [ ]:
rf_new = RandomForestClassifier(n_estimators=76 , random_state=42)
rf_new.fit(A_train,B_train)
rf_new_pred = rf_new.predict(scaler.transform(X))
f1_score(y,rf_new_pred)

In [ ]:
print(confusion_matrix(y,rf_new_pred))
print('\n')
print(classification_report(y,rf_new_pred))

 F1- Score : 69% (rounded to nearest integer value)
 Accuracy (a wrong evaluating metric in this case though) : 96% (rounded to nearest integer value)